This notebook fixes an issue where every player that received a red card continued to be marked with a red card in all subsequent games.

In [1]:
gameweek = 16
total_players = 10945976

In [55]:
import pandas as pd
import numpy as np

import json
import requests
import sys
import pickle
from scipy.stats import poisson

# allow more data columns to be shown than by default
pd.set_option('display.max_columns', 500)

In [3]:
# for some reason, fpl team numbers are otherwise alphabetic, except Leeds comes after Leicester...
teams = ['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton',
        'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Leicester City', 'Leeds United',
         'Liverpool', 'Manchester City', 'Manchester Utd', 'Newcastle Utd',
       'Nottingham Forest', 'Southampton', 'Tottenham', 'West Ham','Wolves']

In [4]:
# fetch FPL data online
fpl_orig_data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
fpl_data = pd.DataFrame(fpl_orig_data['elements'])
#fpl_data.set_index('id',inplace=True)
fpl_data['season'] = '2022-23'
fpl_data['name'] = fpl_data['first_name'] + ' ' + fpl_data['second_name']
# normalize spelling of names
fpl_data['name'] = fpl_data['name'].str.normalize('NFD').str.encode('ascii','ignore').str.decode('ascii')
#fpl_data['name'] = fpl_data['name'].map(player_name_dict).fillna(fpl_data['name'])
position_dict={1:'GK', 2:'DEF', 3:'MID', 4:'FWD'}
fpl_data['position'] = fpl_data['element_type'].map(position_dict)
# change team number to team name
fpl_data['team'] = [teams[i-1] for i in fpl_data['team']]
fpl_data['GW'] = gameweek
fpl_data['value'] = fpl_data['now_cost']/10.0
fpl_data['selected'] = (fpl_data['selected_by_percent'].astype(float) / 100) * total_players
fpl_data['transfers_balance'] = fpl_data['transfers_in_event'] - fpl_data['transfers_out_event']
fpl_data

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,season,name,position,GW,value,selected,transfers_balance
0,100.0,100.0,58822,0,0,-3,3,0,2,0.5,0.5,1,Cédric,0.0,1,False,,2022-09-16T13:30:06.551108Z,42,58822.jpg,1.0,Alves Soares,0.1,False,None,a,Arsenal,3,2,6131,474,26003,490,0.0,0.5,Cédric,28,0,0,0,0,0,0,0,0,0,0,0,5,2.4,1.6,0.0,0.4,0,0.00000,0.00979,0.00979,0.04350,425,159,388,142,477,194,431,159,NaN,,NaN,,NaN,,0.00000,0.0,0.03147,0.03147,0.13982,0.00,544,157,105,62,376,141,400,149,0.00000,0.00000,2022-23,Cedric Alves Soares,DEF,16,4.2,10945.976,-16
1,NaN,NaN,84450,0,0,1,-1,1,3,0.5,0.5,1,Granit,0.0,3,False,,None,51,84450.jpg,4.4,Xhaka,3.6,False,None,a,Arsenal,3,62,743396,68973,525125,94381,0.0,12.2,Xhaka,1166,3,3,6,11,0,0,0,2,0,0,7,253,291.8,278.5,228.0,79.8,14,1.49000,1.48360,2.97360,11.21090,54,21,36,28,63,38,40,29,NaN,,3.0,,NaN,,0.11501,0.0,0.11451,0.22952,0.86534,0.85,171,99,489,183,42,20,86,27,1.08062,0.46312,2022-23,Granit Xhaka,MID,16,5.1,394055.136,-25408
2,100.0,100.0,153256,0,0,-3,3,1,3,0.5,0.5,1,Mohamed,0.0,4,False,,2022-08-30T11:30:06.278675Z,42,153256.jpg,1.3,Elneny,0.8,False,None,a,Arsenal,3,4,167345,25780,218016,17693,0.0,1.0,Elneny,94,0,0,0,1,0,0,0,0,0,0,0,21,4.2,5.3,0.0,1.0,1,0.00000,0.04431,0.04431,0.83280,413,181,365,174,482,187,412,183,NaN,,NaN,,NaN,,0.00000,0.0,0.04242,0.04242,0.79736,0.96,551,284,112,3,343,150,203,64,0.95745,0.00000,2022-23,Mohamed Elneny,MID,16,4.2,87567.808,8087
3,NaN,NaN,156074,0,0,-3,3,0,2,0.5,0.5,0,Rob,0.0,5,False,,None,42,156074.jpg,1.0,Holding,0.1,False,None,a,Arsenal,3,4,6164,568,16322,590,0.0,1.0,Holding,12,0,0,0,0,0,0,0,0,0,0,0,14,2.4,0.1,0.0,0.2,0,0.00000,0.00022,0.00022,0.13740,427,161,428,161,483,198,440,163,NaN,,NaN,,NaN,,0.00000,0.0,0.00165,0.00165,1.03050,0.00,552,163,113,68,379,144,418,155,0.00000,0.00000,2022-23,Rob Holding,DEF,16,4.2,10945.976,-22
4,100.0,100.0,167199,0,0,-2,2,0,3,0.5,0.5,2,Thomas,0.0,6,False,,2022-09-25T09:00:06.484502Z,48,167199.jpg,3.8,Partey,0.5,False,None,a,Arsenal,3,42,85196,9151,102458,8689,0.0,8.8,Partey,959,2,0,7,6,0,0,0,1,0,0,4,191,221.0,141.3,111.0,47.4,11,1.04920,0.56573,1.61493,9.06480,105,44,111,78,150,95,113,71,NaN,,NaN,,NaN,,0.09847,0.0,0.05309,0.15156,0.85071,0.56,260,157,406,129,73,35,242,79,1.03233,0.65693,2022-23,Thomas Partey,MID,16,4.8,54729.880,462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,100.0,0.0,18507,0,0,0,0,0,4,0.0,0.0,0

In [7]:
fpl_data[fpl_data['red_cards']>0][['first_name', 'second_name','web_name']]

,first_name,second_name,web_name
48,Douglas Luiz,Soares de Paulo,Douglas Luiz
185,Conor,Gallagher,Gallagher
190,Kalidou,Koulibaly,Koulibaly
274,Nathaniel,Chalobah,Chalobah
363,Tyler,Adams,Adams
364,Luis,Sinisterra Lucumí,Sinisterra
393,Darwin,Núñez Ribeiro,Darwin
412,João,Cancelo,Cancelo
594,Emerson,Leite de Souza Junior,Emerson Royal
640,Nélson,Cabral Semedo,Semedo


In [11]:
fpl_data[fpl_data['red_cards']>1][['first_name', 'second_name','web_name']]

,first_name,second_name,web_name


In [9]:
# fetch player_dict 
with open('../../data/player_name_dict.txt', 'r') as f:
    player_name_dict_as_text = f.read()
    player_name_dict = json.loads(player_name_dict_as_text)
#display(player_name_dict)

In [15]:
player_data = pd.read_csv('../../data/data_for_projections.csv', index_col=0)
player_data = player_data[player_data['season']=='2022-23'].copy()
player_data = player_data.rename(columns={'GW':'gameweek', 'opp_team_name':'opp_team'})
player_data['name'] = player_data['name'].map(player_name_dict).fillna(player_data['name'])
player_data

,creativity,opp_team,saves,goals_conceded,gameweek,penalties_missed,value,season,position,penalties_saved,yellow_cards,goals_scored,clean_sheets,was_home,name,ict_index,transfers_in,transfers_balance,assists,minutes,own_goals,transfers_out,kickoff_time,selected,red_cards,threat,bps,influence,bonus,total_points,General_#,General_Nation,General_Pos,General_Age,General_Min,Performance_Gls,Performance_Ast,Performance_PK,Performance_PKatt,Performance_Sh,Performance_SoT,Performance_CrdY,Performance_CrdR,Performance_Touches,Performance_Press,Performance_Tkl,Performance_Int,Performance_Blocks,Expected_xG,Expected_npxG,Expected_xA,SCA_SCA,SCA_GCA,Passes_Cmp,Passes_Att,Passes_Cmp%,Passes_Prog,Carries_Carries,Carries_Prog,Dribbles_Succ,Dribbles_Att,team,Total_Cmp,Total_Att,Total_Cmp%,Total_TotDist,Total_PrgDist,Short_Cmp,Short_Att,Short_Cmp%,Medium_Cmp,Medium_Att,Medium_Cmp%,Long_Cmp,Long_Att,Long_Cmp%,General_Ast,General_xA,General_KP,General_1/3,General_PPA,General_CrsPA,General_Prog,General_Att,Pass Types_Live,Pass Types_Dead,Pass Types_FK,Pass Types_TB,Pass Types_Press,Pass Types_Sw,Pass Types_Crs,Pass Types_CK,Corner Kicks_In,Corner Kicks_Out,Corner Kicks_Str,Height_Ground,Height_Low,Height_High,Body Parts_Left,Body Parts_Right,Body Parts_Head,Body Parts_TI,Body Parts_Other,Outcomes_Cmp,Outcomes_Off,Outcomes_Out,Outcomes_Int,Outcomes_Blocks,Tackles_Tkl,Tackles_TklW,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Att 3rd,Vs Dribbles_Tkl,Vs Dribbles_Att,Vs Dribbles_Tkl%,Vs Dribbles_Past,Pressures_Press,Pressures_Succ,Pressures_%,Pressures_Def 3rd,Pressures_Mid 3rd,Pressures_Att 3rd,Blocks_Blocks,Blocks_Sh,Blocks_ShSv,Blocks_Pass,General_Int,General_Tkl+Int,General_Clr,General_Err,Touches_Touches,Touches_Def Pen,Touches_Def 3rd,Touches_Mid 3rd,Touches_Att 3rd,Touches_Att Pen,Touches_Live,Dribbles_Succ%,Dribbles_#Pl,Dribbles_Megs,Carries_TotDist,Carries_PrgDist,Carries_1/3,Carries_CPA,Carries_Mis,Carries_Dis,Receiving_Targ,Receiving_Rec,Receiving_Rec%,Receiving_Prog,Performance_2CrdY,Performance_Fls,Performance_Fld,Performance_Off,Performance_Crs,Performance_TklW,Performance_PKwon,Performance_PKcon,Performance_OG,Performance_Recov,Aerial Duels_Won,Aerial Duels_Lost,Aerial Duels_Won%,General_xAG,Pass Types_TI,Dribbles_Mis,Dribbles_Dis,fpl_position_GK,fpl_position_DEF,fpl_position_MID,fpl_position_FWD,fbref_position_GK,fbref_position_LB,fbref_position_CB,fbref_position_RB,fbref_position_WB,fbref_position_DM,fbref_position_LM,fbref_position_CM,fbref_position_RM,fbref_position_AM,fbref_position_LW,fbref_position_RW,fbref_position_FW,team_xG,team_xGA,team_xG_ewa_5_shifted_0,team_xG_ewa_10_shifted_0,team_xG_ewa_20_shifted_0,team_xG_ewa_40_shifted_0,team_xGA_ewa_5_shifted_0,team_xGA_ewa_10_shifted_0,team_xGA_ewa_20_shifted_0,team_xGA_ewa_40_shifted_0,opp_team_xG,opp_team_xGA,opp_team_xG_ewa_5_shifted_0,opp_team_xG_ewa_10_shifted_0,opp_team_xG_ewa_20_shifted_0,opp_team_xG_ewa_40_shifted_0,opp_team_xGA_ewa_5_shifted_0,opp_team_xGA_ewa_10_shifted_0,opp_team_xGA_ewa_20_shifted_0,opp_team_xGA_ewa_40_shifted_0,xPoints,games_in_past_5_days_shifted_0,games_in_past_10_days_shifted_0,games_in_past_20_days_shifted_0,games_in_past_40_days_shifted_0,selected_shifted_0,transfers_balance_shifted_0,value_shifted_0,transfers_balance_%_shifted_0,transfers_balance_%_ewa5_shifted_0,transfers_balance_%_ewa10_shifted_0,value_change_1_shifted_0,value_change_2_shifted_0,value_change_5_shifted_0,goal_to_xg_ratio_ewa_40_shifted0,total_points_ewa_5_shifted_0,xPoints_ewa_5_shifted_0,assists_ewa_5_shifted_0,bonus_ewa_5_shifted_0,bps_ewa_5_shifted_0,clean_sheets_ewa_5_shifted_0,creativity_ewa_5_shifted_0,goals_conceded_ewa_5_shifted_0,goals_scored_ewa_5_shifted_0,ict_index_ewa_5_shifted_0,influence_ewa_5_shifted_0,minutes_ewa_5_shifted_0,own_goals_ewa_5_shifted_0,penalties_missed_ewa_5_shifted_0,penalties_saved_ewa_5_shifted_0,saves_ewa_5_shifted_0,threat_ewa_5_shifted_0,yellow_cards_ewa_5_shifted_0,Performance_Sh_ewa_5_shifted_0,Performance_SoT_ewa_5_shifted_0,Performance_Touches

In [19]:
aux = player_data.loc[player_data['name'].str.contains('Douglas Luiz')].copy()
aux['red_cards_cumulative'] = aux['red_cards'].cumsum()
aux.loc[aux['red_cards_cumulative']>1, 'red_cards'] = 0
player_data.loc[player_data['name'].str.contains('Douglas Luiz'), 'red_cards'] = aux['red_cards']
player_data.loc[player_data['name'].str.contains('Douglas Luiz'), 'red_cards']

38759    0
39342    0
39624    0
39918    0
40187    0
40468    0
40668    0
40952    0
41240    0
41507    1
41780    0
42064    0
42342    0
42622    0
Name: red_cards, dtype: int64

In [25]:
my_name = 'Gallagher'
aux = player_data.loc[player_data['name'].str.contains(my_name)].copy()
#np.unique(aux['name'])
aux['red_cards_cumulative'] = aux['red_cards'].cumsum()
aux.loc[aux['red_cards_cumulative']>1, 'red_cards'] = 0
player_data.loc[player_data['name'].str.contains(my_name), 'red_cards'] = aux['red_cards']
player_data.loc[player_data['name'].str.contains(my_name), 'red_cards']

38819    0
39109    0
39399    0
39687    1
40245    0
40722    0
41013    0
41296    0
41563    0
42120    0
42399    0
42680    0
Name: red_cards, dtype: int64

In [27]:
my_name = 'Koulibaly'
aux = player_data.loc[player_data['name'].str.contains(my_name)].copy()
#np.unique(aux['name'])
aux['red_cards_cumulative'] = aux['red_cards'].cumsum()
aux.loc[aux['red_cards_cumulative']>1, 'red_cards'] = 0
player_data.loc[player_data['name'].str.contains(my_name), 'red_cards'] = aux['red_cards']
player_data.loc[player_data['name'].str.contains(my_name), 'red_cards']

38822    0
39112    0
39401    1
39977    0
40249    0
41015    0
41299    0
41566    0
42682    0
Name: red_cards, dtype: int64

In [29]:
my_name = 'Chalobah'
aux = player_data.loc[player_data['name'].str.contains(my_name)].copy()
#np.unique(aux['name'])
aux['red_cards_cumulative'] = aux['red_cards'].cumsum()
aux.loc[aux['red_cards_cumulative']>1, 'red_cards'] = 0
player_data.loc[player_data['name'].str.contains(my_name), 'red_cards'] = aux['red_cards']
player_data.loc[player_data['name'].str.contains(my_name), 'red_cards']

39722    0
40007    0
40760    1
39682    0
41009    0
41293    0
41560    0
41836    0
42116    0
42395    0
42676    0
Name: red_cards, dtype: int64

In [32]:
my_name = 'Tyler Adams'
aux = player_data.loc[player_data['name'].str.contains(my_name)].copy()
np.unique(aux['name'])
aux['red_cards_cumulative'] = aux['red_cards'].cumsum()
aux.loc[aux['red_cards_cumulative']>1, 'red_cards'] = 0
player_data.loc[player_data['name'].str.contains(my_name), 'red_cards'] = aux['red_cards']
player_data.loc[player_data['name'].str.contains(my_name), 'red_cards']

38892    0
39186    0
39473    0
39762    0
40042    0
40320    0
40799    0
41089    0
41372    0
41639    0
42193    0
42473    0
42755    1
Name: red_cards, dtype: int64

In [34]:
my_name = 'Sinisterra'
aux = player_data.loc[player_data['name'].str.contains(my_name)].copy()
np.unique(aux['name'])
aux['red_cards_cumulative'] = aux['red_cards'].cumsum()
aux.loc[aux['red_cards_cumulative']>1, 'red_cards'] = 0
player_data.loc[player_data['name'].str.contains(my_name), 'red_cards'] = aux['red_cards']
player_data.loc[player_data['name'].str.contains(my_name), 'red_cards']

39187    0
39474    0
39763    0
40043    0
40321    0
40800    1
41373    0
41640    0
41915    0
Name: red_cards, dtype: int64

In [36]:
my_name = 'Darwin'
aux = player_data.loc[player_data['name'].str.contains(my_name)].copy()
#np.unique(aux['name'])
aux['red_cards_cumulative'] = aux['red_cards'].cumsum()
aux.loc[aux['red_cards_cumulative']>1, 'red_cards'] = 0
player_data.loc[player_data['name'].str.contains(my_name), 'red_cards'] = aux['red_cards']
player_data.loc[player_data['name'].str.contains(my_name), 'red_cards']

38906    0
39200    1
40336    0
40814    0
41103    0
41387    0
41655    0
42206    0
42487    0
42769    0
Name: red_cards, dtype: int64

In [38]:
my_name = 'Cancelo'
aux = player_data.loc[player_data['name'].str.contains(my_name)].copy()
np.unique(aux['name'])
aux['red_cards_cumulative'] = aux['red_cards'].cumsum()
aux.loc[aux['red_cards_cumulative']>1, 'red_cards'] = 0
player_data.loc[player_data['name'].str.contains(my_name), 'red_cards'] = aux['red_cards']
player_data.loc[player_data['name'].str.contains(my_name), 'red_cards']

38912    0
39208    0
39492    0
39784    0
40062    0
40342    0
40549    0
40820    0
41107    0
41390    0
41932    0
42211    0
42492    1
42774    0
Name: red_cards, dtype: int64

In [43]:
my_name = 'Semedo'
aux = player_data.loc[player_data['name'].str.contains(my_name)].copy()
np.unique(aux['name'])
aux['red_cards_cumulative'] = aux['red_cards'].cumsum()
aux.loc[aux['red_cards_cumulative']>1, 'red_cards'] = 0
player_data.loc[player_data['name'].str.contains(my_name), 'red_cards'] = aux['red_cards']
player_data.loc[player_data['name'].str.contains(my_name), 'red_cards']

39306    0
39589    0
39884    0
40153    0
40633    0
40919    0
41205    0
41488    0
41746    0
42029    0
42308    0
42590    1
42866    0
Name: red_cards, dtype: int64

In [44]:
my_name = 'Collins'
aux = player_data.loc[player_data['name'].str.contains(my_name)].copy()
np.unique(aux['name'])
aux['red_cards_cumulative'] = aux['red_cards'].cumsum()
aux.loc[aux['red_cards_cumulative']>1, 'red_cards'] = 0
player_data.loc[player_data['name'].str.contains(my_name), 'red_cards'] = aux['red_cards']
player_data.loc[player_data['name'].str.contains(my_name), 'red_cards']

39023    0
39313    0
39596    0
39889    0
40160    0
40440    0
40639    1
41751    0
42034    0
42313    0
42596    0
42872    0
Name: red_cards, dtype: int64

In [48]:
my_name = 'Costa'
aux = player_data.loc[player_data['name'].str.contains(my_name)].copy()
np.unique(aux['name'])

array([], dtype=object)

In [51]:
model_path = f"../../models/logistic_regression_for_bonus_points.pkl"
with open(model_path, "rb") as f:
    clf = pickle.load(f)

In [52]:
def calculate_xPoints(x,fpl_positions_ordered,clf):
    
    clean_sheet_points = np.array([4,4,1,0])
    goal_points = np.array([6,6,5,4])

    # calculate expexted points
    points_played = np.array([1 if x['minutes']>0 else 0])
    points_played_over_60 = np.array([1 if x['minutes']>=60 else 0])
    points_xG = np.matmul(x[fpl_positions_ordered].values, goal_points) * x['Expected_xG']
    points_xA = x['Expected_xA'] * 3
    clean_sheet_probability = np.array(poisson.pmf(0,x['team_xGA']))
    points_clean_sheet = [np.matmul(x[fpl_positions_ordered].values, clean_sheet_points) *\
                          clean_sheet_probability if x['minutes']>=60 else 0]
    points_saves = x['saves'] // 3
    points_penalty_saves = x['penalties_saved'] * 5 * 0.21 #points for save times approx. probability of penalty save
    penalty_for_potential_penalty_miss = x['Performance_PKatt'] * (-2*0.21)
    # estimate bonus points
    y_pred_prob = clf.predict_proba(np.array(x['bps']).reshape(-1, 1))
    points_bonus = np.matmul(y_pred_prob, np.array([0,1,2,3]).reshape((4,1)))

    # penalty for possible points deductions based on goals conceded
    xGA = x['team_xGA']
    # calculate penalty
    xGA_conceded_penalty = -(poisson.pmf(2,xGA)+poisson.pmf(3,xGA))-(poisson.pmf(4,xGA)+poisson.pmf(5,xGA))-(poisson.pmf(6,xGA)+poisson.pmf(7,xGA))-(poisson.pmf(8,xGA)+poisson.pmf(9,xGA)-(poisson.pmf(10,xGA)+poisson.pmf(11,xGA)))
    # apply penalty only to GK and DEF
    xGA_conceded_penalty = np.matmul(x[fpl_positions_ordered].values, np.array([1,1,0,0])) * xGA_conceded_penalty
    # scale penalty with playing time
    xGA_conceded_penalty = (x['minutes'] / 90) * xGA_conceded_penalty

    penalty_for_cards = [-3 if x['red_cards']==1 else -1 if x['yellow_cards']==1 else 0]
    penalty_for_own_goal = -2 * x['own_goals']

    # add up all point components
    total_points = float(points_played + points_played_over_60 + points_xG + points_xA + points_clean_sheet + points_saves +\
                    points_penalty_saves + penalty_for_potential_penalty_miss + points_bonus + xGA_conceded_penalty +\
                    penalty_for_cards + penalty_for_own_goal)
    
    return total_points

In [53]:
fpl_positions_ordered = [ 'fpl_position_GK', 'fpl_position_DEF', 'fpl_position_MID', 'fpl_position_FWD']

In [56]:
player_data['xPoints'] = player_data.apply(lambda x: calculate_xPoints(x,fpl_positions_ordered,clf), axis=1)
player_data['xPoints'].head(5)

38999    2.112500
39289    4.223747
39570    2.424369
39867    5.538400
40140    4.113599
Name: xPoints, dtype: float64

In [57]:
player_data['xPoints'].tail(5)

42597    1.000127
42683    2.668629
42714    1.400127
42861    2.818263
42876    0.848510
Name: xPoints, dtype: float64

In [58]:
player_data.loc[player_data['name'].str.contains('Darwin'), 'xPoints']

38906    8.327932
39200    0.400000
40336    3.900089
40814    1.400063
41103    4.501224
41387    3.000089
41655    3.217072
42206    4.400257
42487    4.812241
42769    9.229782
Name: xPoints, dtype: float64

In [60]:
player_data_original = pd.read_csv('../../data/data_for_projections.csv', index_col=0)
player_data_original.shape

(42773, 447)

In [61]:
player_data_original.loc[player_data.index, 'xPoints'] = player_data['xPoints']
player_data_original.shape

(42773, 447)

In [62]:
player_data_original.to_csv('../../data/data_for_projections.csv')

In [ ]:
# also should clean up red cards themselves... oh well